<style>
* {
    font-family: consolas,'Microsoft YaHei';
}
p {
    line-height: 1.5em;
    font-size: 20px;
}
.gap {
    display: inline-block;
    padding: 0 10px;
}
</style>
---

<center>
<h1>第八课</h1>
</center>

## **主题**：验证码识别

## 纲要
### 一. 验证码的由来及其作用
### 二. 验证码的种类
### 三. 图形验证码的识别
### 四. 极验滑动验证码的识别
---
 

<center><h2>一. 验证码的由来及其作用</h2></center>


<p>全自动区分计算机和人类的公开图灵测试（英语：Completely Automated Public Turing test to tell Computers and Humans Apart，简称CAPTCHA），俗称验证码，是一种区分用户是机器或人类的公共全自动程序。在CAPTCHA测试中，作为服务器的计算机会自动生成一个问题由用户来解答。这个问题可以由计算机生成并评判，但是必须只有人类才能解答。由于机器无法解答CAPTCHA的问题，回答出问题的用户即可视为人类。（维基百科-验证码）</p>

<p>CAPTCHA技术广泛用于网站的留言板以及注册账号。许多网站为了防止有人利用计算机程序大量在留言板上张贴广告或其他垃圾消息，因此会放置CAPTCHA要求留言者必需输入图片上所显示的文数字或是算术题才可完成留言。而一些网络上的交易系统（例如：订票系统、网络银行）为了避免被计算机程序以暴力破解大量尝试交易也会有CAPTCHA的机制。因此在一般互联网应用中，使用验证码成了一个常规的反爬虫策略。</p>


<center><h2>二. 验证码的种类</h2></center>

<p>常见的验证码主要有：短信验证码、语音电话验证码、图形图像验证码、行为手势验证码等几种类型。其中，短信验证码与语音电话验证码安全性最强，从软件程序角度几乎无破解可能，但其成本相对较高；对于图形图像验证码，简单的有字符辨认类，复杂情况需要人类的记忆理解能力、逻辑思维能力参与（如逻辑问答、物理目标辨认等）；对于行为手势验证码，需要鼠标或手指的拖动、点击、长按等交互行为。</p>

<center><img src="image/graphic1.png" /></center>
<center><img src="image/graphic2.png" /><div class="gap"></div><img src="image/behavier3.png" /></center>
<center><img src="image/behavier1.png" /><div class="gap"></div><img src="image/behavier2.png" /></center>


<center><h2>三. 图形验证码的识别</h2></center>

<p>这里的图形验证码是最简单的字符识别的情形，不涉及物理目标识别。针对这一类验证码，一般利用 OCR 技术进行识别。</p>

In [ ]:
import tesserocr
from PIL import Image

image = Image.open('code2.jpg')

image = image.convert('L')
threshold = 127
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)

image = image.point(table, '1')
result = tesserocr.image_to_text(image)
print(result)

<center><h2>四. 极验滑动验证码的识别</h2></center>

<p>极验验证码是行为手势类验证码的一种，它需要拖动拼合滑块才可以完成验证。极验验证码广泛应用于直播视频、金融服务、电子商务、游戏娱乐、政府企业等各大类型网站。目前极验验证码已更新到了 3.0 版本，全球已有十几万家企业在使用，其每天服务响应超过数亿次。</p>

<p>对于极验 3.0，首先需要点击按钮进行智能验证，如果验证不通过，则会弹出滑动验证的窗口，随后需要拖动滑块拼合图像进行验证，验证之后会生成三个加密参数，参数随后通过表单提交到后台，后台还会进行一次验证。同时，它增加了机器轨迹识别，匀速移动、随机速度移动等方法都不可行，只有完全模拟人的移动轨迹——先加速后减速的过程才可以通过验证。</p>

<p>为避开破解加密参数的复杂过程，我们可以直接使用Selenium进行浏览器模拟。接下来要解决的难点有二：一是如何定位滑块的起点和终点；二是如何模拟鼠标滑动时先加速后减速的过程。</p>

In [4]:
import time
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

EMAIL = 'cqc@cuiqingcai.com'
PASSWORD = ''
BORDER = 6
INIT_LEFT = 60

def browserwraper(*args):
    opts = Options()
    # opts.add_argument('--headless')
    opts.add_argument('--disable-gpu')
    for arg in args:
        opts.add_argument(arg)
    return webdriver.Chrome(executable_path=r'D:\ENV\chromedriver.exe', options=opts)


class CrackGeetest():
    def __init__(self):
        self.url = 'https://account.geetest.com/login'
        self.browser = browserwraper()
        self.wait = WebDriverWait(self.browser, 30)
        self.email = EMAIL
        self.password = PASSWORD
    
    def __del__(self):
        self.browser.close()
    
    def get_geetest_button(self):
        """
        获取初始验证按钮
        :return:
        """
        button = self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'geetest_radar_tip')))
        return button
    
    def get_position(self):
        """
        获取验证码位置
        :return: 验证码位置元组
        """
        img = self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'geetest_canvas_img')))
        time.sleep(2)
        location = img.location
        size = img.size
        top, bottom, left, right = location['y'], location['y'] + size['height'], location['x'], location['x'] + size[
            'width']
        return (top, bottom, left, right)
    
    def get_screenshot(self):
        """
        获取网页截图
        :return: 截图对象
        """
        screenshot = self.browser.get_screenshot_as_png()
        screenshot = Image.open(BytesIO(screenshot))
        return screenshot
    
    def get_slider(self):
        """
        获取滑块
        :return: 滑块对象
        """
        slider = self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'geetest_slider_button')))
        return slider
    
    def get_geetest_image(self, name='captcha.png'):
        """
        获取验证码图片
        :return: 图片对象
        """
        top, bottom, left, right = self.get_position()
        print('验证码位置', top, bottom, left, right)
        screenshot = self.get_screenshot()
        captcha = screenshot.crop((left, top, right, bottom))
        captcha.save(name)
        return captcha
    
    def open(self):
        """
        打开网页输入用户名密码
        :return: None
        """
        self.browser.get(self.url)
        email = self.wait.until(EC.presence_of_element_located((By.ID, 'email')))
        password = self.wait.until(EC.presence_of_element_located((By.ID, 'password')))
        email.send_keys(self.email)
        password.send_keys(self.password)
    
    def get_gap(self, image1, image2):
        """
        获取缺口偏移量
        :param image1: 不带缺口图片
        :param image2: 带缺口图片
        :return:
        """
        left = 60
        for i in range(left, image1.size[0]):
            for j in range(image1.size[1]):
                if not self.is_pixel_equal(image1, image2, i, j):
                    left = i
                    return left
        return left
    
    def is_pixel_equal(self, image1, image2, x, y):
        """
        判断两个像素是否相同
        :param image1: 图片1
        :param image2: 图片2
        :param x: 位置x
        :param y: 位置y
        :return: 像素是否相同
        """
        # 取两个图片的像素点
        pixel1 = image1.load()[x, y]
        pixel2 = image2.load()[x, y]
        threshold = 60
        if abs(pixel1[0] - pixel2[0]) < threshold and abs(pixel1[1] - pixel2[1]) < threshold and abs(
                pixel1[2] - pixel2[2]) < threshold:
            return True
        else:
            return False
    
    def get_track(self, distance):
        """
        根据偏移量获取移动轨迹
        :param distance: 偏移量
        :return: 移动轨迹
        """
        # 移动轨迹
        track = []
        # 当前位移
        current = 0
        # 减速阈值
        mid = distance * 4 / 5
        # 计算间隔
        t = 0.2
        # 初速度
        v = 0
        
        while current < distance:
            if current < mid:
                # 加速度为正2
                a = 2
            else:
                # 加速度为负3
                a = -3
            # 初速度v0
            v0 = v
            # 当前速度v = v0 + at
            v = v0 + a * t
            # 移动距离x = v0t + 1/2 * a * t^2
            move = v0 * t + 1 / 2 * a * t * t
            # 当前位移
            current += move
            # 加入轨迹
            track.append(round(move))
        return track
    
    def move_to_gap(self, slider, track):
        """
        拖动滑块到缺口处
        :param slider: 滑块
        :param track: 轨迹
        :return:
        """
        ActionChains(self.browser).click_and_hold(slider).perform()
        for x in track:
            ActionChains(self.browser).move_by_offset(xoffset=x, yoffset=0).perform()
        time.sleep(0.5)
        ActionChains(self.browser).release().perform()
    
    def login(self):
        """
        登录
        :return: None
        """
        submit = self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'login-btn')))
        submit.click()
        time.sleep(10)
        print('登录成功')
    
    def crack(self):
        # 输入用户名密码
        self.open()
        # 点击验证按钮
        button = self.get_geetest_button()
        button.click()
        # 获取验证码图片
        image1 = self.get_geetest_image('captcha1.png')
        # 点按呼出缺口
        slider = self.get_slider()
        slider.click()
        # 获取带缺口的验证码图片
        image2 = self.get_geetest_image('captcha2.png')
        # 获取缺口位置
        gap = self.get_gap(image1, image2)
        print('缺口位置', gap)
        # 减去缺口位移
        gap -= BORDER
        # 获取移动轨迹
        track = self.get_track(gap)
        print('滑动轨迹', track)
        # 拖动滑块
        self.move_to_gap(slider, track)
        
        success = self.wait.until(
            EC.text_to_be_present_in_element((By.CLASS_NAME, 'geetest_success_radar_tip_content'), '验证成功'))
        print(success)
        
        # 失败后重试
        if not success:
            self.crack()
        else:
            self.login()


if __name__ == '__main__':
    crack = CrackGeetest()
    crack.crack()

TimeoutException: Message: 
